In [61]:
def preprocess_text(text):
    """Preprocess text by keeping all characters as literals"""
    # Convert escape sequences to literal characters
    processed = text.encode('unicode_escape').decode('utf-8')
    return processed

In [62]:
def caesar_cipher(text, alphabet, shift, mode='encrypt', case_strategy='strict', ignore_foreign=False):
    """
    Encrypts or decrypts text using the Caesar cipher.
    
    Args:
        ignore_foreign (bool): If True, foreign characters not in alphabet are ignored
    """
    result = ""
    
    # Create case-specific alphabets if needed
    if case_strategy in ['maintain', 'ignore']:
        lower_alphabet = alphabet.lower()
        upper_alphabet = alphabet.upper()
    
    for char in text:
        if case_strategy == 'strict':
            if char in alphabet:
                index = alphabet.index(char)
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                result += alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
        elif case_strategy == 'maintain':
            if char.lower() in lower_alphabet:
                index = lower_alphabet.index(char.lower())
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                if char.isupper():
                    result += upper_alphabet[new_index]
                else:
                    result += lower_alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
        elif case_strategy == 'ignore':
            char_lower = char.lower()
            if char_lower in lower_alphabet:
                index = lower_alphabet.index(char_lower)
                if mode == 'encrypt':
                    new_index = (index + shift) % len(alphabet)
                else:
                    new_index = (index - shift) % len(alphabet)
                result += lower_alphabet[new_index]
            elif not ignore_foreign:
                result += char
                
    return result

In [63]:
def encode_key(key, alphabet):
    """Encodes the key to a string using the given alphabet (base conversion)."""
    base = len(alphabet)
    if key == 0:
        return alphabet[0]
    
    digits = []
    while key:
        digits.append(alphabet[key % base])
        key //= base
    return "".join(digits[::-1])  # Reverse to get correct order

def decode_key(encoded_key, alphabet):
    """Decodes the key from a string to an integer."""
    base = len(alphabet)
    key = 0
    for char in encoded_key:
        key = key * base + alphabet.index(char)
    return key

In [64]:
def encrypt_with_key(text, alphabet, shift, case_strategy='strict', ignore_foreign=False):
    """Encrypts text and encodes the key within the ciphertext."""
    text = preprocess_text(text)
    encrypted_text = caesar_cipher(text, alphabet, shift, mode='encrypt', 
                                 case_strategy=case_strategy, 
                                 ignore_foreign=ignore_foreign)
    encoded_key = encode_key(shift, alphabet)
    return f"{encoded_key}|{encrypted_text}"

def decrypt_with_key(encrypted_message, alphabet, case_strategy='strict', ignore_foreign=False):
    """Decrypts a message containing an encoded key."""
    try:
        encoded_key, encrypted_text = encrypted_message.split("|", 1)
        shift = decode_key(encoded_key, alphabet)
        return caesar_cipher(encrypted_text, alphabet, shift, mode='decrypt', 
                           case_strategy=case_strategy,
                           ignore_foreign=ignore_foreign)
    except ValueError:
        return "Invalid encrypted message format."

In [65]:
def shift_alphabet(alphabet, shift):
    return alphabet[shift:] + alphabet[:shift]

In [66]:
# Example usage:
alphabet = r"abcdefghijklmnopqrstuvwxyz"
text = "If he had anything confidential to say, he wrote it in cipher, that is, by so changing the order of the letters of the alphabet, that not a word could be made out."
shift = 7

# Test Strict
encrypted_maintain = encrypt_with_key(text, alphabet, shift, case_strategy='strict')
print(f"Encrypted (strict): {encrypted_maintain}")
decrypted_maintain = decrypt_with_key(encrypted_maintain, alphabet, case_strategy='strict')
print(f"Decrypted (strict): {decrypted_maintain}")


# Test maintain case
encrypted_maintain = encrypt_with_key(text, alphabet, shift, case_strategy='maintain')
print(f"Encrypted (maintain case): {encrypted_maintain}")
decrypted_maintain = decrypt_with_key(encrypted_maintain, alphabet, case_strategy='maintain')
print(f"Decrypted (maintain case): {decrypted_maintain}")

# Test ignore case
encrypted_ignore = encrypt_with_key(text, alphabet, shift, case_strategy='ignore')
print(f"Encrypted (ignore case): {encrypted_ignore}")
decrypted_ignore = decrypt_with_key(encrypted_ignore, alphabet, case_strategy='ignore')
print(f"Decrypted (ignore case): {decrypted_ignore}")



Encrypted (strict): h|Im ol ohk hufaopun jvumpkluaphs av zhf, ol dyval pa pu jpwoly, aoha pz, if zv johunpun aol vykly vm aol slaalyz vm aol hswohila, aoha uva h dvyk jvbsk il thkl vba.
Decrypted (strict): If he had anything confidential to say, he wrote it in cipher, that is, by so changing the order of the letters of the alphabet, that not a word could be made out.
Encrypted (maintain case): h|Pm ol ohk hufaopun jvumpkluaphs av zhf, ol dyval pa pu jpwoly, aoha pz, if zv johunpun aol vykly vm aol slaalyz vm aol hswohila, aoha uva h dvyk jvbsk il thkl vba.
Decrypted (maintain case): If he had anything confidential to say, he wrote it in cipher, that is, by so changing the order of the letters of the alphabet, that not a word could be made out.
Encrypted (ignore case): h|pm ol ohk hufaopun jvumpkluaphs av zhf, ol dyval pa pu jpwoly, aoha pz, if zv johunpun aol vykly vm aol slaalyz vm aol hswohila, aoha uva h dvyk jvbsk il thkl vba.
Decrypted (ignore case): if he had anything confidentia

In [67]:
# With foreign characters included
encrypted = encrypt_with_key(text, alphabet, shift, case_strategy='maintain', ignore_foreign=False)
print(f"Encrypted (with foreign): {encrypted}")
decrypted = decrypt_with_key(encrypted, alphabet, case_strategy='maintain', ignore_foreign=False)
print(f"Decrypted (with foreign): {decrypted}")

# With foreign characters ignored
encrypted = encrypt_with_key(text, alphabet, shift, case_strategy='maintain', ignore_foreign=True)
print(f"Encrypted (ignore foreign): {encrypted}")
decrypted = decrypt_with_key(encrypted, alphabet, case_strategy='maintain', ignore_foreign=True)
print(f"Decrypted (ignore foreign): {decrypted}")

Encrypted (with foreign): h|Pm ol ohk hufaopun jvumpkluaphs av zhf, ol dyval pa pu jpwoly, aoha pz, if zv johunpun aol vykly vm aol slaalyz vm aol hswohila, aoha uva h dvyk jvbsk il thkl vba.
Decrypted (with foreign): If he had anything confidential to say, he wrote it in cipher, that is, by so changing the order of the letters of the alphabet, that not a word could be made out.
Encrypted (ignore foreign): h|Pmolohkhufaopunjvumpkluaphsavzhfoldyvalpapujpwolyaohapzifzvjohunpunaolvyklyvmaolslaalyzvmaolhswohilaaohauvahdvykjvbskilthklvba
Decrypted (ignore foreign): Ifhehadanythingconfidentialtosayhewroteitincipherthatisbysochangingtheorderofthelettersofthealphabetthatnotawordcouldbemadeout
